In [19]:
# for audio recognizion
import pyaudio as audio
import speech_recognition as sr
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk import RegexpParser
# for stable diffusion
import os
import random
import PIL
import tensorflow as tf
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
from datetime import datetime
from math import ceil, floor

# create a list of stopwords to ignore...
stopwords = set(['shan', 'same', "wasn't", "she's", 'they', 'off', 
                 "needn't", "weren't", 'as', 'some', 'and', 'from', 'other', 
                 "shouldn't", "shan't", 'to', 'does', 'was', 'has', 'so', 'himself', 
                 'do', 'below', "doesn't", "that'll", 'its', 'these', 'are', 'more', 
                 'aren', 'all', 'whom', 'shouldn', 'too', 'over', "you've", 'him', 'o', 
                 'his', 'be', "you'll", 'out', 'against', 'most', 'if', 'hasn', 'own', 
                 's', 'what', 'theirs', 'or', "it's", 'will', "don't", 'is', 'been', 
                 'who', 'yourselves', 'her', 'did', 'the', 'up', 'there', 'ourselves', 
                 'during', 'mightn', "you'd", 'further', 'very', 'those', 'for', 'but', 
                 'an', 'in', 'nor', "mightn't", 've', 'both', 'until', 'isn', 'ain', 
                 "didn't", 'than', 'themselves', 'myself', "couldn't", 'now', 'herself', 
                 'any', 'by', "wouldn't", 'about', 'after', 'here', 'doesn', 'a', 'which', 
                 'd', 'y', 'were', 'couldn', "aren't", 'i', 'then', 'being', 'just', 'our', 
                 "haven't", 't', 'wouldn', 're', "mustn't", 'while', 'with', 'only', 'under', 
                 'ma', 'again', 'can', 'ours', 'through', "hadn't", 'when', 'hers', "isn't", 
                 'of', 'few', 'my', 'had', 'before', 'where', 'wasn', "should've", 'she', 
                 'your', 'haven', 'weren', 'on', 'have', 'he', 'between', 'me', 'down', 
                 'should', 'mustn', 'their', 'am', 'above', 'll', 'such', 'why', 'no', 
                 'you', 'it', 'because', 'into', 'm', "you're", 'that', 'itself', 'not', 
                 'hadn', "won't", 'we', 'don', 'doing', 'won', 'them', 'this', "hasn't", 
                 'how', 'at', 'needn', 'once', 'having', 'yours', 'each', 'yourself', 'didn'])
# create stemmer object to reduce words to their root forms...
stemmer = PorterStemmer()
# create a queue of the last 100 words identified by the program
recent_text_q = []
# activate macbook microphone stream
## figure out what audio device our microphone is
print(sr.Microphone.list_microphone_names())
index = 0
print("should be the internal microphone: ", sr.Microphone.list_microphone_names()[index])
# create a microphone object
internal_mic = sr.Microphone(device_index = index)
# create a speech recognition object
recognizer = sr.Recognizer()
return_str = ""

# create a basic naming root for any files that we create and want to save during this session
time_str = datetime.now().strftime("%Y_%m_%d_%H_")
output_name = os.getcwd() + "/output_images/" + time_str
img_num = 0
print(output_name)

words = []
nouns = []
verbs = []
adjectives = []
determiners = []
recent_text_q = []

def addSimplePhraise():
    # generate multiple phraises, then determine best one using nltk
    return "{} {} {} {} {}".format(randomNoun(), randomAdj(), randomNoun(), randomVerb(), randomNoun())

def randomNoun():
    if len(nouns):
        return nouns[random.randint(0, len(nouns) - 1)]
    return ""

def randomVerb():
    if len(verbs):
        return verbs[random.randint(0, len(verbs) - 1)]
    return ""

def randomAdj():
    if len(adjectives):
        return adjectives[random.randint(0, len(adjectives) - 1)]
    return ""

def randomDeterminer():
    if len(determiners):
        return determiners[random.randint(0, len(determiners))]
    return ""

def fifoIn(lst, val, max_len):
    # check if item needs to be removed
    if len(lst) >= max_len:
        lst.pop(0)
    lst.append(val)
    return lst

def getStrFromTuple(lst):
    r = ""
    for l in lst:
        r.join(l[0]).join(" ")
    return r

def getStrFromList(lst):
    print(lst)
    r = ""
    for i in range(len(lst)):
        print(i)
        r.join(lst[i]).join(" ")
        print(r)
    return r

def generatePromptString():
    return "{}, {}, {}".format(addSimplePhraise(), addSimplePhraise(), addSimplePhraise())
    

['MacBook Pro Microphone', 'MacBook Pro Speakers', 'Microsoft Teams Audio', 'ZoomAudioDevice']
should be the internal microphone:  MacBook Pro Microphone
/Users/nathan/workspace/neural_art_bots/output_images/2023_02_19_17_


In [20]:
##############################
only_unique_words = True
max_q_len = 100
###############33

gender = 'm' # as oppose to  'f'
vnum = str(random.randint(1, 5))
voice = '{}{}'.format(gender, vnum)
pitch = str(random.randint(10, 90))

batch = 2
seed = None
steps = 15

nprompt_string = "dogs, cars, people"

# 0.7 is the default
# Higher guidance scale encourages to generate 
# images that are closely linked to the text prompt, 
# usually at the expense of lower image quality.
uncon_guide_scale = 0.3

# model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)
# MQ model for standard operation speed
model = keras_cv.models.StableDiffusion(img_width=640, img_height=480, jit_compile=True)
# LQ and fast version of the model
# model = keras_cv.models.StableDiffusion(img_width=96, img_height=96, jit_compile=False)
# HQ and slow version
#model = keras_cv.models.StableDiffusion(img_width=512, img_height=512, jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [24]:
def listenForWords(recent_text_q, words, nouns, verbs, adjectives, determiners):
    # capture audio from the microphone
    with internal_mic as source:
        # adjust for background noise to increase success rate
        recognizer.adjust_for_ambient_noise(source)
        # identify any spoken words in the audio
        print("Speech Recognizer Enabled");
        audio = recognizer.listen(source)
        print("audio exported from source")
        raw_string = "the the "
        try:
            raw_string = recognizer.recognize_google(audio)
        except:
            print(" .")

        print("{} of tokenized words returned from google: {}".format(type(raw_string), raw_string))
        # remove words from string

        return_str = raw_string.split()
        wn = len(return_str)
        # remove any words in the stopwords
                # append spoken words to the running FIFO of all words
            # if append to buffer according to type of grammer
        words = [i for i in return_str if i not in stopwords]
        for word in words:
            recent_text_q = fifoIn(recent_text_q, word, max_q_len)
        print("{} words removed from stopwords".format(wn - len(words)))
        if only_unique_words is True:
            words = [i for i in return_str if i not in recent_text_q]
            print("{} words removed from priorwords".format(wn - len(words)))
        # append the word type to the word string, returnig a tuple (str, type)
        words = pos_tag(words)
        # reconstruct a string to be sent to a stable diffusion network
        print("{} word tags tuples : {}".format(len(words), words))
        # TODO - BUG - because we are not operating on the "words" object we are running into an issue
        for word in words:
            print("word : {}".format(word))
            if word[1].startswith("NN"):
                nouns = fifoIn(nouns, word[0], max_q_len)
            elif word[1].startswith("VB"):
                verbs = fifoIn(verbs, word[0], max_q_len)
            elif word[1].startswith("JJ"):
                adjectives = fifoIn(adjectives, word[0], max_q_len)
            elif word[1].startswith("DT"):
                determiners = fifoIn(determiners, word[0], max_q_len)
            else:
                print("word not categorized: {}".format(word))
            prompt_string = ""
        print("{} nouns are saved: {}".format(len(nouns), nouns))
        print("{} verbs are saved: {}".format(len(verbs), verbs))
        print("{} adjectives are saved: {}".format(len(adjectives), adjectives))
        print("{} determiners are saved: {}".format(len(determiners), determiners))
        # create a chunk grammar statement
        """
            According to the rule you created, your chunks:
            Start with an optional (?) determiner ('DT')
            Can have any number (*) of adjectives (JJ)
            End with a noun (<NN>)
        grammar = "NP: {<DT>?<JJ>*<NN>}"
        chunk_parser = RegexpParser(grammar)
        tree = chunk_parser.parse(words_tags)
        """
        print("{} identified words: ".format(len(recent_text_q)),
              recent_text_q)

In [28]:

listenForWords(recent_text_q, words, nouns, verbs, adjectives, determiners)

Speech Recognizer Enabled
audio exported from source
result2:
{   'alternative': [   {   'confidence': 0.94409424,
                           'transcript': "I don't really understand if no "
                                         'words are identified then whether '
                                         'they show up in the list'},
                       {   'transcript': "I don't really understand if no "
                                         'words are identified then why do '
                                         'they show up in the list'},
                       {   'transcript': "I don't really understand if "
                                         "nobody's are identified then whether "
                                         'they show up in the list'},
                       {   'transcript': "I don't really understand if no "
                                         'words are identified then whether '
                                         'they show up in th

In [29]:
prompt_string = generatePromptString()
print(prompt_string)

don't  don't are don't, don't  don't are don't, don't  don't are don't


## Text to Speech using espeak in OS

In [30]:
command = 'espeak -s 180 -v {} -p {} "{}"\n'.format(voice, pitch, prompt_string)
print("using espeak to say the following command {}".format(prompt_string))
os.system(command)

using espeak to say the following command don't  don't are don't, don't  don't are don't, don't  don't are don't


11

In [31]:
#########################################################
images = model.text_to_image(
    prompt_string,
    negative_prompt=nprompt_string,
    seed=seed,
    num_steps=steps,
    batch_size=batch,
    unconditional_guidance_scale=uncon_guide_scale,
)

keras.backend.clear_session()  # Clear session to preserve memory
plot_images(images)
img_num = save_images(images)

2023-02-19 18:02:05.631753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 18:02:12.251524: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fa425a9b240 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-02-19 18:02:12.251580: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2023-02-19 18:02:12.251881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild Te

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
 4/16 [======>.......................] - ETA: 6:15

In [ ]:
# cool, lets now put all the chunks together =)
